<a href="https://colab.research.google.com/github/Feranie/Hierarchical-Classification-Project/blob/main/Inconsistency%20Rate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd  # Importa biblioteca pandas para manipulação de dados tabulares
import numpy as np  # Importa numpy para operações matemáticas e arrays
import re  # Importa regex para processamento de strings e padrões
import time  # Importa time para medir tempos de execução
import random  # Importa random para geração de números aleatórios
from collections import defaultdict, Counter  # Importa estruturas de dados especializadas
from typing import List, Tuple, Dict, Any  # Importa tipos para anotações de tipo

# --- Leitura de Arquivo ARFF ---
def read_arff_file(file_path):
    """
    Lê arquivo ARFF e retorna DataFrame pandas.

    Args:
        file_path: Caminho do arquivo ARFF

    Returns:
        DataFrame pandas com os dados
    """
    data = []  # Inicializa lista vazia para armazenar linhas de dados
    attributes = []  # Inicializa lista vazia para armazenar nomes dos atributos
    current_section = None  # Variável para rastrear em qual seção do ARFF estamos

    # Abre o arquivo em modo leitura com codificação UTF-8
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:  # Para cada linha do arquivo
            line = line.strip()  # Remove espaços em branco no início e fim
            if not line or line.startswith('%'):  # Se linha vazia ou comentário
                continue  # Pula para próxima linha

            if '@relation' in line.lower():  # Se encontrou declaração @relation
                current_section = 'relation'  # Marca seção atual como 'relation'
            elif '@attribute' in line.lower():  # Se encontrou declaração @attribute
                current_section = 'attribute'  # Marca seção atual como 'attribute'
                # Usa regex para extrair nome do atributo da linha
                match = re.match(r'@attribute\s+([^\s]+)\s+.*', line, re.IGNORECASE)
                if match:  # Se regex encontrou correspondência
                    attributes.append(match.group(1))  # Adiciona nome do atributo à lista
            elif '@data' in line.lower():  # Se encontrou declaração @data
                current_section = 'data'  # Marca seção atual como 'data'
            elif current_section == 'data':  # Se estamos na seção de dados
                # Usa regex para dividir linha por vírgulas (mantendo valores entre aspas juntos)
                values = re.findall(r'[^,]+(?:,(?=[^,]$))?', line)
                # Remove aspas e espaços de cada valor
                values = [v.strip('" ') for v in values]
                if len(values) == len(attributes):  # Se número de valores bate com atributos
                    data.append(values)  # Adiciona linha aos dados

    # Converte lista de dados para DataFrame pandas com nomes das colunas
    return pd.DataFrame(data, columns=attributes)

# --- Salvamento de Arquivo ARFF ---
def save_to_arff(df, file_path, relation_name="filtered_data"):
    """
    Salva DataFrame no formato de arquivo ARFF.

    Args:
        df: DataFrame pandas a ser salvo
        file_path: Caminho onde salvar o arquivo
        relation_name: Nome da relação no ARFF
    """
    # Abre arquivo em modo escrita com codificação UTF-8
    with open(file_path, 'w', encoding='utf-8') as f:
        # Escreve declaração @relation
        f.write(f"@relation {relation_name}\n\n")

        # Para cada coluna do DataFrame
        for column in df.columns:
            unique_values = df[column].unique()  # Obtém valores únicos da coluna
            # Se todos os valores são strings (atributo categórico)
            if all(isinstance(val, str) for val in unique_values):
                vals = ','.join(sorted(set(unique_values)))  # Junta valores únicos separados por vírgula
                # Escreve declaração de atributo categórico
                f.write(f"@attribute {column} {{{vals}}}\n")
            else:  # Se atributo numérico
                # Escreve declaração de atributo numérico
                f.write(f"@attribute {column} numeric\n")

        f.write("\n@data\n")  # Escreve marcador de início dos dados
        # Para cada linha do DataFrame
        for _, row in df.iterrows():
            # Escreve linha com valores separados por vírgula
            f.write(",".join(map(str, row)) + "\n")

class ConsistencyMeasure:
    """
    Implementação da medida de consistência (taxa de inconsistência) conforme descrito em:
    M. Dash, H. Liu / Artificial Intelligence 151 (2003) 155–176

    Esta métrica avalia a qualidade de um subconjunto de atributos medindo
    quantas instâncias têm o mesmo padrão de atributos mas classes diferentes.
    """

    def __init__(self, data: pd.DataFrame, class_column: str):
        """
        Inicializa o calculador de medida de consistência.

        Args:
            data: DataFrame contendo o dataset
            class_column: Nome da coluna de classe/alvo
        """
        self.data = data  # Armazena o DataFrame completo
        self.class_column = class_column  # Armazena nome da coluna de classe
        # Cria lista de colunas de atributos (todas exceto a classe)
        self.feature_columns = [col for col in data.columns if col != class_column]
        self.total_instances = len(data)  # Armazena número total de instâncias

    def calculate_inconsistency_rate(self, feature_subset: List[str]) -> float:
        """
        Calcula a taxa de inconsistência para um dado subconjunto de atributos.

        Taxa de Inconsistência (IR) = Número de instâncias inconsistentes / Total de instâncias

        Uma instância é inconsistente quando:
        - Tem o mesmo padrão de valores de atributos que outras instâncias
        - Mas pertence a uma classe diferente da classe majoritária desse padrão

        Args:
            feature_subset: Lista de nomes de atributos a considerar

        Returns:
            Taxa de inconsistência (IR) para o subconjunto de atributos
        """
        if not feature_subset:  # Se subconjunto vazio
            return 0.0  # Retorna 0 (sem atributos = sem inconsistência mensurável)

        # Valida se todos os atributos do subconjunto existem no dataset
        invalid_features = [f for f in feature_subset if f not in self.feature_columns]
        if invalid_features:  # Se há atributos inválidos
            # Lança erro com lista de atributos inválidos
            raise ValueError(f"Atributos inválidos: {invalid_features}")

        # Agrupa instâncias por padrão (combinação de valores de atributos)
        patterns = self._group_by_pattern(feature_subset)

        # Calcula contagem de inconsistências para cada padrão
        total_inconsistency_count = 0  # Inicializa contador total

        # Para cada padrão único e suas instâncias
        for pattern, instances in patterns.items():
            # Calcula inconsistências deste padrão específico
            inconsistency_count = self._calculate_pattern_inconsistency(instances)
            total_inconsistency_count += inconsistency_count  # Acumula no total

        # Calcula taxa de inconsistência = inconsistências / total de instâncias
        inconsistency_rate = total_inconsistency_count / self.total_instances
        return inconsistency_rate  # Retorna taxa (valor entre 0 e 1)

    def _group_by_pattern(self, feature_subset: List[str]) -> Dict[Tuple, List[Any]]:
        """
        Agrupa instâncias por seu padrão (combinações de valores de atributos).

        Exemplo:
        Se feature_subset = ['idade', 'salário']
        Padrão (25, 3000) pode ter instâncias com classes [1, 1, 0]
        Padrão (30, 4000) pode ter instâncias com classes [0, 0]

        Args:
            feature_subset: Lista de nomes de atributos

        Returns:
            Dicionário mapeando padrões para listas de rótulos de classe
        """
        # Cria dicionário que automaticamente cria listas vazias para novas chaves
        patterns = defaultdict(list)

        # Para cada linha no dataset
        for _, row in self.data.iterrows():
            # Cria tupla com valores dos atributos selecionados
            # Exemplo: (25, 3000) para atributos ['idade', 'salário']
            pattern = tuple(row[feature] for feature in feature_subset)
            class_label = row[self.class_column]  # Obtém rótulo de classe desta instância
            patterns[pattern].append(class_label)  # Adiciona classe à lista deste padrão

        return patterns  # Retorna dicionário padrão -> lista de classes

    def _calculate_pattern_inconsistency(self, class_labels: List[Any]) -> int:
        """
        Calcula contagem de inconsistências para um único padrão.

        Exemplo:
        Se um padrão tem classes [1, 1, 1, 0, 0]:
        - Classe majoritária: 1 (aparece 3 vezes)
        - Inconsistências: 2 (as duas instâncias com classe 0)

        Args:
            class_labels: Lista de rótulos de classe para instâncias com o mesmo padrão

        Returns:
            Contagem de inconsistências para este padrão
        """
        if len(class_labels) <= 1:  # Se há 1 ou 0 instâncias com este padrão
            return 0  # Não há inconsistência (precisa de pelo menos 2 para conflito)

        # Conta ocorrências de cada rótulo de classe
        # Exemplo: Counter([1, 1, 1, 0, 0]) retorna {1: 3, 0: 2}
        label_counts = Counter(class_labels)

        # Encontra contagem máxima (classe mais frequente)
        # Exemplo: max({1: 3, 0: 2}.values()) retorna 3
        max_count = max(label_counts.values())

        # Contagem de inconsistências = total de instâncias - instâncias da classe majoritária
        # Exemplo: 5 instâncias - 3 (maioria) = 2 inconsistências
        inconsistency_count = len(class_labels) - max_count

        return inconsistency_count  # Retorna número de instâncias inconsistentes

    def analyze_pattern_details(self, feature_subset: List[str]) -> Dict:
        """
        Fornece análise detalhada de padrões e suas inconsistências.

        Útil para entender quais padrões são problemáticos e por quê.

        Args:
            feature_subset: Lista de nomes de atributos a analisar

        Returns:
            Dicionário com análise detalhada dos padrões
        """
        # Agrupa instâncias por padrão
        patterns = self._group_by_pattern(feature_subset)

        # Inicializa dicionário de análise com estatísticas
        analysis = {
            'total_patterns': len(patterns),  # Número total de padrões únicos
            'inconsistent_patterns': 0,  # Contador de padrões com inconsistências
            'pattern_details': [],  # Lista para detalhes de cada padrão
            'total_inconsistency_count': 0  # Contador total de inconsistências
        }

        # Para cada padrão e suas classes
        for pattern, class_labels in patterns.items():
            label_counts = Counter(class_labels)  # Conta ocorrências de cada classe
            # Calcula inconsistências deste padrão
            inconsistency_count = self._calculate_pattern_inconsistency(class_labels)

            # Cria dicionário com informações sobre este padrão
            pattern_info = {
                'pattern': pattern,  # Valores dos atributos (ex: (25, 3000))
                'total_instances': len(class_labels),  # Quantas instâncias têm este padrão
                'class_distribution': dict(label_counts),  # Distribuição de classes {1: 3, 0: 2}
                'inconsistency_count': inconsistency_count,  # Número de inconsistências
                'is_inconsistent': inconsistency_count > 0  # Boolean: tem inconsistência?
            }

            analysis['pattern_details'].append(pattern_info)  # Adiciona à lista de detalhes
            analysis['total_inconsistency_count'] += inconsistency_count  # Acumula total

            if inconsistency_count > 0:  # Se este padrão tem inconsistências
                analysis['inconsistent_patterns'] += 1  # Incrementa contador

        # Calcula taxa de inconsistência total
        analysis['inconsistency_rate'] = analysis['total_inconsistency_count'] / self.total_instances

        return analysis  # Retorna dicionário com análise completa

    def compare_feature_subsets(self, feature_subsets: List[List[str]]) -> pd.DataFrame:
        """
        Compara taxas de inconsistência entre múltiplos subconjuntos de atributos.

        Útil para avaliar qual combinação de atributos é mais consistente.

        Args:
            feature_subsets: Lista de listas de atributos para comparar

        Returns:
            DataFrame com resultados da comparação ordenados por IR
        """
        results = []  # Inicializa lista para resultados

        # Para cada subconjunto de atributos
        for i, subset in enumerate(feature_subsets):
            ir = self.calculate_inconsistency_rate(subset)  # Calcula IR
            # Adiciona resultado à lista
            results.append({
                'subset_id': i,  # ID do subconjunto
                'features': subset,  # Lista de atributos
                'num_features': len(subset),  # Quantidade de atributos
                'inconsistency_rate': ir,  # Taxa de inconsistência
                'consistency_score': 1 - ir  # Score de consistência (quanto maior, melhor)
            })

        # Converte lista para DataFrame e ordena por IR (menor primeiro = melhor)
        return pd.DataFrame(results).sort_values('inconsistency_rate')

class RandomRestartHillClimbing:
    """
    Algoritmo Random Restart Hill Climbing para encontrar subconjunto ótimo de atributos
    que minimiza a taxa de inconsistência (IR).

    O algoritmo:
    1. Inicia de múltiplos pontos aleatórios (restarts)
    2. Para cada início, executa hill climbing (busca local)
    3. Aceita mudanças que melhoram o IR
    4. Para quando não há mais melhorias
    5. Retorna o melhor resultado de todos os restarts
    """

    def __init__(self, consistency_measure: ConsistencyMeasure, max_restarts=10,
                 max_iterations=100, min_features=1):
        """
        Inicializa otimizador RRHC.

        Args:
            consistency_measure: Instância de ConsistencyMeasure
            max_restarts: Número máximo de reinícios aleatórios
            max_iterations: Máximo de iterações por restart
            min_features: Número mínimo de atributos a manter
        """
        self.cm = consistency_measure  # Armazena calculador de consistência
        self.max_restarts = max_restarts  # Armazena número de restarts
        self.max_iterations = max_iterations  # Armazena máximo de iterações
        self.min_features = min_features  # Armazena mínimo de atributos
        self.all_features = self.cm.feature_columns.copy()  # Copia lista de todos atributos

    def optimize(self, verbose=True):
        """
        Executa Random Restart Hill Climbing para encontrar subconjunto ótimo.

        Args:
            verbose: Imprimir informações de progresso

        Returns:
            Dicionário com resultados da otimização
        """
        best_subset = None  # Inicializa melhor subconjunto como None
        best_ir = float('inf')  # Inicializa melhor IR como infinito (pior possível)
        best_restart = -1  # Inicializa índice do melhor restart
        all_results = []  # Lista para armazenar resultados de todos restarts

        # Para cada restart
        for restart in range(self.max_restarts):
            if verbose:  # Se modo verbose ativado
                print(f"\n--- Restart {restart + 1}/{self.max_restarts} ---")

            # Gera subconjunto inicial aleatório
            initial_size = random.randint(self.min_features, len(self.all_features))  # Tamanho aleatório
            current_subset = random.sample(self.all_features, initial_size)  # Seleciona atributos aleatórios
            current_ir = self.cm.calculate_inconsistency_rate(current_subset)  # Calcula IR inicial

            if verbose:
                print(f"Subconjunto inicial: {len(current_subset)} atributos, IR: {current_ir:.4f}")

            # Hill climbing a partir deste ponto inicial
            result = self._hill_climb(current_subset, current_ir, verbose)
            all_results.append(result)  # Adiciona resultado à lista

            # Atualiza melhor global se este restart foi melhor
            if result['final_ir'] < best_ir:
                best_ir = result['final_ir']  # Atualiza melhor IR
                best_subset = result['final_subset'].copy()  # Atualiza melhor subconjunto
                best_restart = restart  # Marca qual restart foi o melhor

            if verbose:
                print(f"Final: {len(result['final_subset'])} atributos, IR: {result['final_ir']:.4f}")

        # Retorna dicionário com todos os resultados
        return {
            'best_subset': best_subset,  # Melhor subconjunto encontrado
            'best_ir': best_ir,  # Melhor IR encontrado
            'best_restart': best_restart,  # Qual restart foi o melhor
            'all_results': all_results,  # Resultados de todos os restarts
            'total_evaluations': sum(r['evaluations'] for r in all_results)  # Total de avaliações
        }

    def _hill_climb(self, initial_subset, initial_ir, verbose=False):
        """
        Executa hill climbing a partir de um subconjunto inicial.

        Aceita mudanças que diminuem o IR (melhoram a consistência).
        Para quando não há mais melhorias possíveis.
        """
        current_subset = initial_subset.copy()  # Copia subconjunto inicial
        current_ir = initial_ir  # Armazena IR inicial
        iteration = 0  # Contador de iterações
        evaluations = 1  # Contador de avaliações (conta avaliação inicial)
        improvements = 0  # Contador de melhorias

        # Loop até máximo de iterações
        while iteration < self.max_iterations:
            iteration += 1  # Incrementa contador
            improved = False  # Flag para indicar se houve melhoria

            # Gera todos os vizinhos (adiciona/remove um atributo)
            neighbors = self._generate_neighbors(current_subset)

            # Avalia todos os vizinhos
            for neighbor in neighbors:
                if len(neighbor) < self.min_features:  # Se vizinho tem menos que mínimo
                    continue  # Pula este vizinho

                neighbor_ir = self.cm.calculate_inconsistency_rate(neighbor)  # Calcula IR do vizinho
                evaluations += 1  # Incrementa contador de avaliações

                # Aceita se melhor (IR menor)
                if neighbor_ir < current_ir:
                    current_subset = neighbor.copy()  # Atualiza subconjunto atual
                    current_ir = neighbor_ir  # Atualiza IR atual
                    improved = True  # Marca que houve melhoria
                    improvements += 1  # Incrementa contador de melhorias

                    # Log a cada 10 iterações
                    if verbose and iteration % 10 == 0:
                        print(f"  Iteração {iteration}: {len(current_subset)} atributos, IR: {current_ir:.4f}")
                    break  # Aceita primeira melhoria (greedy)

            if not improved:  # Se não houve melhoria
                break  # Para o hill climbing (ótimo local encontrado)

        # Retorna dicionário com resultados deste hill climb
        return {
            'final_subset': current_subset,  # Subconjunto final
            'final_ir': current_ir,  # IR final
            'iterations': iteration,  # Número de iterações
            'evaluations': evaluations,  # Número de avaliações
            'improvements': improvements  # Número de melhorias
        }

    def _generate_neighbors(self, current_subset):
        """
        Gera subconjuntos vizinhos adicionando/removendo um atributo.

        Vizinhos são subconjuntos que diferem em exatamente um atributo.
        """
        neighbors = []  # Inicializa lista de vizinhos
        current_set = set(current_subset)  # Converte para conjunto para busca rápida

        # Remove um atributo (se possível)
        if len(current_subset) > self.min_features:  # Se há margem para remover
            for feature in current_subset:  # Para cada atributo no subconjunto atual
                # Cria vizinho removendo este atributo
                neighbor = [f for f in current_subset if f != feature]
                neighbors.append(neighbor)  # Adiciona à lista de vizinhos

        # Adiciona um atributo
        # Encontra atributos que NÃO estão no subconjunto atual
        available_features = [f for f in self.all_features if f not in current_set]
        for feature in available_features:  # Para cada atributo disponível
            neighbor = current_subset + [feature]  # Cria vizinho adicionando atributo
            neighbors.append(neighbor)  # Adiciona à lista de vizinhos

        return neighbors  # Retorna lista de todos os vizinhos

def generate_smart_feature_subsets(feature_columns, max_combinations=200, include_large_subsets=True):
    """
    Gera seleção inteligente de subconjuntos de atributos para datasets grandes.
    Usa amostragem estratégica para evitar explosão combinatorial.

    Estratégia:
    1. Inclui todos os atributos individuais
    2. Amostra pares de atributos
    3. Amostra alguns subconjuntos maiores (3, 4, 5, 10, 15, 20)
    4. Adiciona subconjuntos aleatórios de tamanhos variados

    Args:
        feature_columns: Lista de nomes de atributos
        max_combinations: Número máximo de subconjuntos a gerar
        include_large_subsets: Se deve incluir subconjuntos maiores

    Returns:
        Lista de subconjuntos de atributos
    """
    from itertools import combinations  # Importa função para gerar combinações
    import random  # Já importado no topo, mas explicitamente aqui

    feature_subsets = []  # Inicializa lista de subconjuntos
    n_features = len(feature_columns)  # Número total de atributos

    print(f"Geração inteligente de subconjuntos para {n_features} atributos...")

    # Sempre inclui atributos individuais
    print("Adicionando atributos individuais...")
    for feature in feature_columns:  # Para cada atributo
        feature_subsets.append([feature])  # Adiciona como subconjunto de tamanho 1

    remaining_budget = max_combinations - len(feature_subsets)  # Calcula orçamento restante

    # Adiciona pares estrategicamente
    if n_features <= 20:  # Se dataset pequeno
        # Suficientemente pequeno para incluir todos os pares
        print("Adicionando todos os pares...")
        for pair in combinations(feature_columns, 2):  # Gera todas as combinações de 2
            feature_subsets.append(list(pair))  # Adiciona par à lista
            remaining_budget -= 1  # Decrementa orçamento
            if remaining_budget <= 0:  # Se orçamento esgotado
                break  # Para de adicionar
    else:  # Se dataset grande
        # Amostra pares aleatoriamente
        pair_budget = min(100, remaining_budget // 2)  # Reserva até 100 ou metade do orçamento
        print(f"Amostrando {pair_budget} pares...")
        all_pairs = list(combinations(feature_columns, 2))  # Gera todos os pares possíveis
        sampled_pairs = random.sample(all_pairs, min(pair_budget, len(all_pairs)))  # Amostra
        for pair in sampled_pairs:  # Para cada par amostrado
            feature_subsets.append(list(pair))  # Adiciona à lista
        remaining_budget -= len(sampled_pairs)  # Decrementa orçamento

    # Adiciona subconjuntos maiores se solicitado e há orçamento
    if include_large_subsets and remaining_budget > 0:
        for size in [3, 4, 5, 10, 15, 20]:  # Para cada tamanho específico
            if size > n_features or remaining_budget <= 0:  # Se tamanho inválido ou sem orçamento
                break  # Para

            # Calcula orçamento para este tamanho
            size_budget = min(20, remaining_budget // (6 - (size - 3)),
                            remaining_budget if size >= 10 else remaining_budget)

            if size_budget > 0:  # Se há orçamento
                print(f"Amostrando {size_budget} subconjuntos de tamanho {size}...")
                all_combinations = list(combinations(feature_columns, size))  # Gera todas combinações
                sample_size = min(size_budget, len(all_combinations))  # Limita ao disponível
                sampled_combinations = random.sample(all_combinations, sample_size)  # Amostra

                for combo in sampled_combinations:  # Para cada combinação amostrada
                    feature_subsets.append(list(combo))  # Adiciona à lista

                remaining_budget -= sample_size  # Decrementa orçamento

    # Adiciona subconjuntos aleatórios de tamanhos variados
    if remaining_budget > 0:
        print(f"Adicionando {remaining_budget} subconjuntos aleatórios...")
        for _ in range(remaining_budget):  # Para o orçamento restante
            size = random.randint(2, min(25, n_features))  # Tamanho aleatório entre 2 e 25
            random_subset = random.sample(feature_columns, size)  # Amostra atributos aleatórios
            feature_subsets.append(random_subset)  # Adiciona à lista

    print(f"Total gerado: {len(feature_subsets)} subconjuntos")
    return feature_subsets  # Retorna lista completa de subconjuntos

# --- Função Principal ---
def main():
    """
    Função principal para carregar dados, otimizar atributos e salvar resultados.

    Fluxo:
    1. Carrega dados do arquivo ARFF
    2. Inicializa medida de consistência
    3. Calcula IR inicial com todos atributos
    4. Gera e avalia subconjuntos inteligentemente
    5. Executa otimização RRHC
    6. Compara resultados e seleciona melhor solução
    7. Salva dataset otimizado
    """
    # Define caminhos dos arquivos
    input_file_path = '/content/GPCR-PrositeTRA0.arff'  # Arquivo de entrada
    output_file_path = '/content/GPCR-PrositeTRA0OptimizedRRHCIR.arff'  # Arquivo de saída

    print("Carregando dados...")  # Log
    start_time = time.time()  # Marca tempo de início
    data = read_arff_file(input_file_path)  # Carrega dados do ARFF
    load_time = time.time() - start_time  # Calcula tempo de carregamento
    # Imprime estatísticas de carregamento
    print(f"Carregados {len(data)} instâncias com {len(data.columns)-1} atributos em {load_time:.2f}s")

    # Assume que última coluna é a classe (ajuste se necessário)
    class_column = data.columns[-1]  # Obtém nome da última coluna
    print(f"Coluna de classe: {class_column}")  # Log
    # Imprime distribuição de classes
    print(f"Distribuição de classes: {data[class_column].value_counts().to_dict()}")

    # Inicializa medida de consistência
    print("\nInicializando medida de consistência...")
    cm = ConsistencyMeasure(data, class_column)  # Cria instância

    # Calcula taxa de inconsistência inicial com todos os atributos
    print("Calculando taxa de inconsistência inicial...")
    initial_ir = cm.calculate_inconsistency_rate(cm.feature_columns)  # Calcula IR
    # Imprime IR inicial
    print(f"Taxa de inconsistência inicial (todos os {len(cm.feature_columns)} atributos): {initial_ir:.4f}")

    print("\n" + "="*60)
    print("FASE 1: AVALIAÇÃO INTELIGENTE DE SUBCONJUNTOS")
    print("="*60)

    # Gera subconjuntos inteligentemente baseado no tamanho do dataset
    print("Dataset grande: geração inteligente de subconjuntos")
    # Gera até 500 subconjuntos diferentes para avaliar
    feature_subsets = generate_smart_feature_subsets(cm.feature_columns, max_combinations=500)

    # Avalia subconjuntos com rastreamento de progresso
    print("\nAvaliando subconjuntos gerados...")
    results = []  # Lista para armazenar resultados
    start_eval = time.time()  # Marca tempo de início da avaliação

    # Para cada subconjunto gerado
    for i, subset in enumerate(feature_subsets):
        try:
            ir = cm.calculate_inconsistency_rate(subset)  # Calcula IR do subconjunto
            # Adiciona resultado à lista
            results.append({
                'subset_id': i,  # ID do subconjunto
                'features': subset,  # Lista de atributos
                'num_features': len(subset),  # Número de atributos
                'inconsistency_rate': ir,  # Taxa de inconsistência
                'consistency_score': 1 - ir  # Score de consistência
            })

            # Atualiza progresso a cada 50 avaliações
            if (i + 1) % 50 == 0:
                elapsed = time.time() - start_eval  # Tempo decorrido
                avg_time = elapsed / (i + 1)  # Tempo médio por avaliação
                remaining = len(feature_subsets) - (i + 1)  # Avaliações restantes
                eta = remaining * avg_time / 60  # Tempo estimado em minutos
                best_so_far = min(r['inconsistency_rate'] for r in results)  # Melhor IR até agora
                # Imprime progresso
                print(f"Progresso: {((i+1)/len(feature_subsets)*100):.1f}% ({i+1}/{len(feature_subsets)}) - "
                      f"ETA: {eta:.1f} min - Melhor IR: {best_so_far:.4f}")
        except Exception as e:  # Se erro ao avaliar
            print(f"Erro ao avaliar subconjunto {i}: {e}")  # Log do erro
            continue  # Pula para próximo

    # Converte resultados para DataFrame e ordena por IR
    subset_comparison = pd.DataFrame(results).sort_values('inconsistency_rate')
    print(f"\nAvaliação concluída: {len(subset_comparison)} subconjuntos testados")

    # Mostra top 10 resultados
    print("\nTop 10 melhores subconjuntos:")
    top_subsets = subset_comparison.head(10)  # Pega os 10 melhores
    for idx, row in top_subsets.iterrows():  # Para cada um
        # Calcula melhoria percentual
        improvement = ((initial_ir - row['inconsistency_rate']) / initial_ir * 100)
        # Imprime informações
        print(f"  {idx+1}: {row['num_features']} atributos, IR: {row['inconsistency_rate']:.4f} "
              f"(melhoria: {improvement:.1f}%)")

    # Obtém melhor subconjunto da avaliação
    best_exhaustive_subset = subset_comparison.iloc[0]['features']  # Melhor subconjunto
    best_exhaustive_ir = subset_comparison.iloc[0]['inconsistency_rate']  # Melhor IR

    print(f"\nMelhor subconjunto encontrado:")
    print(f"  Número de atributos: {len(best_exhaustive_subset)}")
    print(f"  Taxa de inconsistência: {best_exhaustive_ir:.4f}")
    print(f"  Melhoria: {((initial_ir - best_exhaustive_ir) / initial_ir * 100):.2f}%")

    # Inicializa e executa Random Restart Hill Climbing
    print("\n" + "="*60)
    print("FASE 2: OTIMIZAÇÃO RRHC")
    print("="*60)
    print("Iniciando otimização Random Restart Hill Climbing...")

    # Cria instância do otimizador com parâmetros
    optimizer = RandomRestartHillClimbing(
        consistency_measure=cm,  # Passa medida de consistência
        max_restarts=10,  # 10 reinícios aleatórios
        max_iterations=50,  # Máximo 50 iterações por restart
        min_features=1  # Mínimo 1 atributo
    )

    optimization_start = time.time()  # Marca tempo de início
    result = optimizer.optimize(verbose=True)  # Executa otimização com logs
    optimization_time = time.time() - optimization_start  # Calcula tempo total

    # Exibe resultados da otimização
    print("\n" + "="*60)
    print("RESULTADOS DA OTIMIZAÇÃO")
    print("="*60)
    print(f"Melhor subconjunto encontrado: {len(result['best_subset'])} atributos")
    print(f"Atributos selecionados: {result['best_subset']}")
    print(f"Taxa de inconsistência otimizada: {result['best_ir']:.4f}")

    # Compara métodos e seleciona melhor solução
    print("\n" + "="*60)
    print("COMPARAÇÃO DOS MÉTODOS")
    print("="*60)
    print(f"Avaliação inteligente:")
    print(f"  Melhor subconjunto: {len(best_exhaustive_subset)} atributos")
    print(f"  Taxa de inconsistência: {best_exhaustive_ir:.4f}")
    print(f"\nRandom Restart Hill Climbing:")
    print(f"  Melhor subconjunto: {len(result['best_subset'])} atributos")
    print(f"  Taxa de inconsistência: {result['best_ir']:.4f}")
    print(f"  Avaliações totais: {result['total_evaluations']}")
    print(f"  Tempo de otimização: {optimization_time:.2f}s")

    # Seleciona melhor resultado entre os dois métodos
    if result['best_ir'] <= best_exhaustive_ir:  # Se RRHC foi melhor ou igual
        print(f"✓ RRHC encontrou solução igual ou melhor!")
        final_best_subset = result['best_subset']  # Usa resultado do RRHC
        final_best_ir = result['best_ir']
    else:  # Se avaliação inteligente foi melhor
        print(f"✓ Avaliação inteligente encontrou melhor solução!")
        # Calcula quanto RRHC poderia melhorar
        improvement_possible = ((result['best_ir'] - best_exhaustive_ir) / result['best_ir'] * 100)
        print(f"  Melhoria possível de RRHC: {improvement_possible:.2f}%")
        final_best_subset = best_exhaustive_subset  # Usa resultado da avaliação
        final_best_ir = best_exhaustive_ir

    # Imprime solução final escolhida
    print(f"\nSOLUÇÃO FINAL:")
    print(f"  Número de atributos selecionados: {len(final_best_subset)}")
    print(f"  Taxa de inconsistência final: {final_best_ir:.4f}")
    # Calcula melhoria em relação ao inicial
    print(f"  Melhoria vs inicial: {((initial_ir - final_best_ir) / initial_ir * 100):.2f}%")
    # Calcula redução percentual de atributos
    reduction = ((len(cm.feature_columns) - len(final_best_subset)) / len(cm.feature_columns) * 100)
    print(f"  Redução de atributos: {reduction:.1f}%")

    # Cria dataset otimizado com o melhor subconjunto
    selected_columns = final_best_subset + [class_column]  # Adiciona coluna de classe
    optimized_data = data[selected_columns]  # Seleciona apenas colunas escolhidas

    # Salva dataset otimizado
    print(f"\nSalvando dataset otimizado...")
    save_to_arff(optimized_data, output_file_path, "optimized_data_RRHC")  # Salva em ARFF
    print(f"Dataset salvo: {output_file_path}")

    # Análise adicional detalhada
    print("\n" + "="*60)
    print("ANÁLISE DETALHADA")
    print("="*60)

    # Análise de padrões para subconjunto otimizado
    print(f"\nAnálise de padrões para o subconjunto otimizado:")
    pattern_analysis = cm.analyze_pattern_details(final_best_subset)  # Analisa padrões
    print(f"  Padrões totais: {pattern_analysis['total_patterns']}")
    print(f"  Padrões inconsistentes: {pattern_analysis['inconsistent_patterns']}")
    # Score de consistência (quanto maior, melhor)
    print(f"  Score de consistência: {(1 - pattern_analysis['inconsistency_rate']):.4f}")

    # Mostra alguns padrões inconsistentes se existirem
    if pattern_analysis['inconsistent_patterns'] > 0:
        print(f"\nExemplos de padrões inconsistentes (primeiros 3):")
        # Filtra apenas padrões inconsistentes e pega os 3 primeiros
        inconsistent_patterns = [detail for detail in pattern_analysis['pattern_details']
                               if detail['is_inconsistent']][:3]
        for i, detail in enumerate(inconsistent_patterns):  # Para cada padrão
            # Imprime informações do padrão
            print(f"  {i+1}. Padrão {detail['pattern']}: {detail['total_instances']} instâncias")
            print(f"     Distribuição de classes: {detail['class_distribution']}")
            print(f"     Número de inconsistências: {detail['inconsistency_count']}")

    # Mostra atributos selecionados
    print(f"\nAtributos selecionados no melhor subconjunto:")
    for i, feature in enumerate(final_best_subset, 1):  # Enumera começando de 1
        print(f"  {i:2d}. {feature}")  # Imprime número e nome do atributo

    # Análise por tamanho de subconjunto
    if len(subset_comparison) > 10:  # Se há dados suficientes
        print(f"\nAnálise por tamanho de subconjunto:")
        size_analysis = {}  # Dicionário para agrupar por tamanho

        # Agrupa IRs por tamanho de subconjunto
        for _, row in subset_comparison.iterrows():
            size = row['num_features']  # Tamanho do subconjunto
            if size not in size_analysis:  # Se tamanho ainda não está no dicionário
                size_analysis[size] = []  # Cria lista vazia
            size_analysis[size].append(row['inconsistency_rate'])  # Adiciona IR à lista

        # Mostra estatísticas para cada tamanho
        for size in sorted(size_analysis.keys())[:10]:  # Primeiros 10 tamanhos
            rates = size_analysis[size]  # Lista de IRs para este tamanho
            avg_rate = np.mean(rates)  # Calcula média
            min_rate = min(rates)  # Encontra mínimo
            # Imprime estatísticas
            print(f"  Tamanho {size:2d}: IR médio = {avg_rate:.4f}, IR mínimo = {min_rate:.4f} "
                  f"({len(rates)} subconjuntos)")

    # Retorna dicionário com todos os resultados para uso posterior
    return {
        'optimized_data': optimized_data,  # DataFrame otimizado
        'best_subset': final_best_subset,  # Melhor subconjunto de atributos
        'best_ir': final_best_ir,  # Melhor IR encontrado
        'initial_ir': initial_ir,  # IR inicial (referência)
        'improvement': ((initial_ir - final_best_ir) / initial_ir * 100),  # Melhoria %
        'subset_comparison': subset_comparison,  # DataFrame com comparações
        'rrhc_result': result  # Resultados detalhados do RRHC
    }

# Ponto de entrada do programa
if __name__ == "__main__":
    # Executa a função principal quando script é executado diretamente
    main()